# Asset management with modifications

A modified version of the Asset Management Problem Taken from the book
J.R. Birge, F. Louveaux, Introduction to Stochastic Programming,
Springer Series in Operations Research and Financial Engineering,
Springer New York, New York, NY, 2011

In [ ]:
using SDDP, HiGHS, Test

function asset_management_stagewise(; cut_type)
    w_s = [1.25, 1.06]
    w_b = [1.14, 1.12]
    Phi = [-1, 5]
    Psi = [0.02, 0.0]

    model = SDDP.MarkovianPolicyGraph(;
        sense = :Max,
        transition_matrices = Array{Float64,2}[
            [1.0]',
            [0.5 0.5],
            [0.5 0.5; 0.5 0.5],
            [0.5 0.5; 0.5 0.5],
        ],
        upper_bound = 1000.0,
        optimizer = HiGHS.Optimizer,
    ) do subproblem, node
        t, i = node
        @variable(subproblem, xs >= 0, SDDP.State, initial_value = 0)
        @variable(subproblem, xb >= 0, SDDP.State, initial_value = 0)
        if t == 1
            @constraint(subproblem, xs.out + xb.out == 55 + xs.in + xb.in)
            @stageobjective(subproblem, 0)
        elseif t == 2 || t == 3
            @variable(subproblem, phi)
            @constraint(
                subproblem,
                w_s[i] * xs.in + w_b[i] * xb.in + phi == xs.out + xb.out
            )
            SDDP.parameterize(subproblem, [1, 2], [0.6, 0.4]) do ω
                JuMP.fix(phi, Phi[ω])
                @stageobjective(subproblem, Psi[ω] * xs.out)
            end
        else
            @variable(subproblem, u >= 0)
            @variable(subproblem, v >= 0)
            @constraint(
                subproblem,
                w_s[i] * xs.in + w_b[i] * xb.in + u - v == 80,
            )
            @stageobjective(subproblem, -4u + v)
        end
    end
    SDDP.train(
        model;
        cut_type = cut_type,
        log_frequency = 10,
        risk_measure = (node) -> begin
            if node[1] != 3
                SDDP.Expectation()
            else
                SDDP.EAVaR(; lambda = 0.5, beta = 0.5)
            end
        end,
    )
    @test SDDP.calculate_bound(model) ≈ 1.278 atol = 1e-3
    return
end

asset_management_stagewise(; cut_type = SDDP.SINGLE_CUT)

asset_management_stagewise(; cut_type = SDDP.MULTI_CUT)